<a href="https://colab.research.google.com/github/ZacharySBrown/vcu-scma440-2021q1/blob/master/labs/Regression_Arima_Errors_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Fetch Data
Press the play button in Google Colab or press Shift+Enter execute the cell below to download and load the data and packages for this assignment

This will setup you environment and load a DataFrame called `bankdata` that contains the bank data from Makridakis Chapter 6.

In [16]:
!pip install --upgrade statsmodels
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.stattools import durbin_watson
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display
from statsmodels.tsa.arima.model import ARIMA
#plt.style.use('dark_background')


from scipy.stats import t, f

%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 16

!curl https://vcu-scma-440.s3.amazonaws.com/data/bankdata.csv > bankdata.csv

bankdata = pd.read_csv('bankdata.csv', parse_dates=['date']).set_index('date').rename(columns={'34': 'mo_34'})

Requirement already up-to-date: statsmodels in /usr/local/lib/python3.7/dist-packages (0.12.2)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1687  100  1687    0     0  13604      0 --:--:-- --:--:-- --:--:-- 13604


# Inspecting the data

Before we get started with any analysis, let's begin by taking a look at the data and plotting the four columns. In the cell below, take a look at the data and plot the time series. It may be useful to plot the `balance` column by itself, and the `aaa` and `mo_34` columns together. 

Remember, to plot a single column, you type:

```
display(plot['my_column'].plot())
```

and to plot two columns, you type:

```
display(plot[['one_column','another_column']].plot())
```

(note the _two_ sets of `[]` when you're specifying two columns)

In [25]:
# YOUR CODE HERE


## Creating Difference Variables

These series appear to be non-stationary in the mean, so we'll begin by removing that non-stationarity using differencing. We can do this using the `.diff()` function: 

```
data['my_diff_column'] = data['my_original_column'].diff()
```

Below we'll carry this out for the `balance`, `aaa`, and `mo_34` columns, naming the new columns containing the differences as `d_balance`, `d_aaa`, and `d_mo_34`. 

**Note here that this introduces a single row with `NaN` in it from taking the first difference, which we'll drop using `data = data.dropna()`**


In [26]:
bankdata['d_balance'] = bankdata['balance'].diff()
bankdata['d_aaa'] = bankdata['aaa'].diff()
bankdata['d_mo_34'] = bankdata['mo_34'].diff()
bankdata = bankdata.dropna()
bankdata.head()

,balance,aaa,mo_34,d_balance,d_aaa,d_mo_34
date,,,,,,
2000-03-01,358774,6.08,5.49,-2443.0,0.08,-0.11
2000-04-01,360271,6.17,5.80,1497.0,0.09,0.31
2000-05-01,360139,6.14,5.61,-132.0,-0.03,-0.19
2000-06-01,362164,6.09,5.28,2025.0,-0.05,-0.33
2000-07-01,362901,5.87,5.19,737.0,-0.22,-0.09


We'll next want to check that this data appears stationary. Plot `d_balance` by itself, and `d_aaa` and `d_mo_34` to make sure they are now stationary in the mean. 

In [27]:
# YOUR CODE HERE


# Regression with ARIMA errors

We'll not attempt to fit a regression model to this data with ARIMA errors. We'll start with a simple `AR(1)` (`ARIMA(1,0,0)`) model for our regression errors. In the cell below, create an ARIMA model using `d_balance` as endog, and `d_aaa` and `d_mo_34` as exog, and set the appropriate value for `order`. Then fit the model (`model.fit()`) and print out the summary of the results. 

In [28]:
# YOUR CODE HERE


If any of the variables in the fit above do not have significant weights (p-value > 0.05), drop those from the fit and rerun the fit.

# Determining Appropriate ARIMA model for errors

Now that we've carried out a regression, we want to check the residuals for any autoregressive patterns and update our choice of ARIMA model for the errors if necessary. Judigin from the value for Ljung-Box Q above, it appears that our residuals don't have any autocorrelation, but we can plot them, as well as their ACF and PACF to double check. In the cell below, plot the residuals with `display(plt.plot(results.resid))`, and then plot the ACF and PACF plots for `results.resid` (remember to set `zero=False`).

In [29]:
# YOUR CODE HERE


# Updating Preliminary ARIMA Model

From the above, we see strong autocorrelations remain up through lag 3 in the PACF. It also appears that we a seasonal autoregressive pattern at lag 12 (with a repetition at 24). We'll update our ARIMA model for the errors to an:
$$ARIMA(3,0,0)(2,0,0)_{12}$$

In the cell below, create a new model, updating the value passed to `order` above, and additionally specifying the `seasonal_order` argument to be `(1,0,0,12)`

In [30]:
# YOUR CODE HERE


Note the much larger p-value now for the Ljung-Box Q, indicating that there is very little autocorrelation remaining in the residuals. 

In the cell below, verify that the observed autocorrelation in the resoduals has been reduced, by plotting the ACF and PACF

In [31]:
# YOUR CODE HERE
